#**load libraries**

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
from shapely.wkt import loads as wkt_loads
import tifffile as tiff
import os
import random
from keras import backend as K
from sklearn.metrics import jaccard_similarity_score
#from sklearn.metrics import jaccard_score
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
from collections import defaultdict
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
import gc
import warnings
warnings.filterwarnings("ignore")
from tifffile import imread, imwrite
from skimage.transform import resize

**load the data**

In [8]:
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        #print(len(batch))
        return tuple(batch)
    
    def __len__(self):
        return len(self.indexes) // self.batch_size

class Dataset:
  
    def __init__(self, images_dir, mask_dir):
        
        self.ids = images_dir
        self.images_fps = images_dir
        self.masks_fps  = mask_dir
    
    def __getitem__(self, i):
        
        # read data
        image = np.load(self.images_fps[i]) 
        mask  = np.load(self.masks_fps[i])

          
        image = np.stack(image, axis=-1).astype('float')
        mask = np.stack(mask, axis=-1).astype('float')

        #image = np.transpose(image, (1,0,2)) 
        #mask = np.transpose(mask, (1,0,2)) 
    
        image = np.transpose(image, (0,2,1)) 
        mask = np.transpose(mask, (0,2,1)) 
  
        return image, mask
      
    def __len__(self):
        return len(self.ids)        

In [9]:
x_test = np.load("/content/drive/My Drive/data_path/x_test.npy")
y_test = np.load("/content/drive/My Drive/data_path/y_test.npy")
test_dataset = Dataset(x_test, y_test)
test_dataloader = Dataloder(test_dataset, batch_size=1)

**pick the random point for testing**

In [10]:
a = rn.randint(0,len(test_dataloader))
image = test_dataloader[a][0]
mask =  test_dataloader[a][1]

In [11]:
num_cls = 10
size = 160
smooth = 1e-12

In [12]:
#https://www.kaggle.com/aamaia/rgb-using-m-bands-example

def adjust_contrast(bands, lower_percent=2, higher_percent=98):
    """
    to adjust the contrast of the image 
    bands is the image 
    """
    out = np.zeros_like(bands).astype(np.float32)
    n = bands.shape[2]
    for i in range(n):
        a = 0  # np.min(band)
        b = 1  # np.max(band)
        c = np.percentile(bands[:, :, i], lower_percent)
        d = np.percentile(bands[:, :, i], higher_percent)
        t = a + (bands[:, :, i] - c) * (b - a) / (d - c)
        t[t < a] = a
        t[t > b] = b
        out[:, :, i] = t

    return out.astype(np.float32)

In [13]:
def jaccard_coef(y_true, y_pred):
    """
    Jaccard Index: Intersection over Union.
    J(A,B) = |A∩B| / |A∪B| 
         = |A∩B| / |A|+|B|-|A∩B|
    """
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    total = K.sum(y_true + y_pred, axis=[0, -1, -2])
    union = total - intersection

    jac = (intersection + smooth) / (union+ smooth)

    return K.mean(jac)

#**SegNet**

In [15]:

##fixing numpy RS
np.random.seed(42)

##fixing tensorflow RS
tf.random.set_seed(32)

##python RS
rn.seed(12)

def SegNet():

    classes= 10
    img_input = Input(shape=(size, size, 8))
    x = img_input

    # Encoder 
    
    x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 23))(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same',  kernel_initializer = tf.keras.initializers.he_normal(seed= 43))(x)
   # x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 32))(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 41))(x)
   # x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 33))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = Dropout(0.5)(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 35))(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 54))(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 39))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    #Decoder
    
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 45))(x)
   # x = BatchNormalization()(x)
    x = Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 41))(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 49))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
      
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 18))(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 21))(x)
    x = BatchNormalization()(x)
    x = Conv2D(classes, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 16))(x)
    x = Dropout(0.25)(x)
  
    x = Activation("softmax")(x)
    
    model = Model(img_input, x)
  
    model.compile(optimizer=Adam(lr=1e-4),loss='binary_crossentropy', metrics=[jaccard_coef])
    return model

In [61]:
def resize_image(image, model):
  """
  to resize the image
  """
  if image.shape == (837,837,8):
    return image

  else:
    resized_data = resize(image, (837,837,8))
    imwrite('resized.tif', resized_data, planarconfig='CONTIG')
    return tiff.imread("resized.tif")

In [77]:
#https://www.kaggle.com/drn01z3/end-to-end-baseline-with-u-net-keras

def prediction(id, model):

    """
    take inputs as image id and trained model 
    and returns predicted mask 
    """
  
    #Read 16-band image
    rgb_img = os.path.join( 'sixteen_band', '{}_M.tif'.format(id))      
    rgb_img = tiff.imread(rgb_img)
    img = np.rollaxis(rgb_img, 0, 3)


    #resize the image according to model architecture
    img = resize_image(img, model)

    #adjust the contrast of the image
    x = adjust_contrast(img)
    
    cnv = np.zeros((960, 960, 8)).astype(np.float32)
    prd = np.zeros((num_cls, 960, 960)).astype(np.float32)
    cnv[:img.shape[0], :img.shape[1], :] = x
     
    for i in range(0, 6):
        line = []
        for j in range(0, 6):
            line.append(cnv[i * size:(i + 1) * size, j * size:(j + 1) * size])
            
  
        x = 2 * np.transpose(line, (0, 3, 1, 2)) - 1
       # x = 2 * np.transpose(line, (0, 1, 2, 3)) - 1

        tmp = model.predict(x, batch_size=4)
       # tmp = np.transpose(tmp,(0,3,1,2))

        for j in range(tmp.shape[0]):
            prd[:, i * size:(i + 1) * size, j * size:(j + 1) * size] = tmp[j]
     
    # thresholds for each class 
    trs = [0.4, 0.1, 0.4, 0.3, 0.3, 0.5, 0.3, 0.6, 0.1, 0.1]
    for i in range(num_cls):
        prd[i] = prd[i] > trs[i]

    return prd[:, :img.shape[0], :img.shape[1]]

In [81]:
def final_fun1(id):
    """
    takes input as image id and 
    returns predicted mask 
    """

    #load the image through input image id 
    rgb_img = os.path.join( 'sixteen_band', '{}_M.tif'.format(id))      
    rgb_img = tiff.imread(rgb_img)
    image = np.rollaxis(rgb_img, 0, 3)
    print(image.shape)

    #load the trained model
    model = SegNet()
    model.load_weights("/content/weights-59-0.4575.hdf5")

    #predict the mask input image id
    mask = prediction(id, model)

    class_list = ["Buildings", "Misc.Manmade structures" ,"Road",\
                  "Track","Trees","Crops","Waterway","Standing water",\
                  "Vehicle Large","Vehicle Small"]
    
    img = np.zeros((image.shape[0],image.shape[1],3))
    img[:,:,0] = image[:,:,4] #red
    img[:,:,1] = image[:,:,2] #green
    img[:,:,2] = image[:,:,1] #blue

    for i in range(num_cls):
        plt.figure(figsize=(25,25))
        ax1 = plt.subplot(131)
        ax1.set_title('image ID:6120_2_0')
        ax1.imshow(adjust_contrast(img))
        ax2 = plt.subplot(132)
        ax2.set_title("predict "+ class_list[i] +" pixels")
        ax2.imshow(mask[i], cmap=plt.get_cmap('gray'))
        plt.show()

In [80]:
final_fun1("6120_2_0")

Output hidden; open in https://colab.research.google.com to view.

In [82]:
def final_fun2(data_point, data_point_label):
    """
    returns jaccard score between actual mask and predicted mask
    """

    image = data_point
    actual_mask = data_point_label

    #load the trained model
    model = SegNet()
    model.load_weights("/content/weights-59-0.4575.hdf5")

    predicted_mask = model.predict(image)
    print(image.shape, predicted_mask.shape)

    score = jaccard_coef(actual_mask, predicted_mask)
    return score

In [83]:
score = final_fun2(image, mask)

(1, 160, 160, 8) (1, 160, 160, 10)


In [84]:
print("The jaccard score on a test data point is", score.numpy())

The jaccard score on a test data point is 0.2884757894539237
